In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Reading data

In [2]:
# read train data
df_orders = pd.read_csv("DATA/orders.csv", sep=";")
df_products = pd.read_csv("DATA/product_attributes.csv", sep=",")
df_products.loc[len(df_products)] = [-1, 1303.3, 2]
df_dists = pd.read_csv("DATA/cities_data.csv", sep=";")

# read test data
df_test = pd.read_csv("DATA/test.csv", sep=";")

len(df_orders)

114276

## cities_data.csv

In [3]:
df_dists = pd.read_csv("DATA/cities_data.csv", sep=";")
df_aux2 = pd.DataFrame({'city_to_name':[],'city_from_name':[], 'city_to_coord':[], 'city_from_coord':[], 'distance':[]})

#Afegim duplicats
df_aux = df_dists.copy()
df_aux['city_from_name'] = df_dists['city_to_name']
df_aux['city_to_name'] = df_dists['city_from_name']
df_aux['city_to_coord'] = df_dists['city_from_coord']
df_aux['city_from_coord'] = df_dists['city_to_coord']


#Afefim ciutat a ella mateixa
df_aux4 = df_dists[['city_from_name', 'city_from_coord']].copy().drop_duplicates()
df_aux5 = df_dists[['city_to_name', 'city_to_coord']].copy().drop_duplicates()
df_aux3 = pd.concat([df_aux4, df_aux5.rename(columns ={'city_to_name':'city_from_name', 'city_to_coord':'city_from_coord'})]).drop_duplicates()

df_aux2['city_from_name'] = df_aux3['city_from_name']
df_aux2['city_from_coord'] = df_aux3['city_from_coord']
df_aux2['city_to_name'] = df_aux3['city_from_name']
df_aux2['city_to_coord'] = df_aux3['city_from_coord']
df_aux2['distance'] = 0


df_dists = pd.concat([df_dists, df_aux])
df_dists = pd.concat([df_dists, df_aux2])

## orders.csv

In [4]:
# Refactor variables
def bool2int(row, col):
    row[col] = int(row[col])
    return row

def clean_cities(row, col):
    if row[col] == "BCN":
        row[col] = "Barcelona"
    if row[col] == "ATHENAS":
        row[col] = "Athens"
    return row

df_orders = df_orders.apply(lambda row: bool2int(row, "late_order"), axis = 1)
df_orders = df_orders.apply(lambda row: clean_cities(row, "origin_port"), axis = 1)

In [5]:
# dummies in categorical columns
col2prefix = {
    "origin_port":"OP",
    "3pl":"3pl",
    "customs_procedures":"CP",
    "logistic_hub":"LH",
    "customer":"customer"
}

for col, prefix in col2prefix.items():
    df = pd.get_dummies(df_orders[col], prefix=prefix, drop_first=False)
    df = pd.DataFrame(df_orders[col]).join(df).drop_duplicates()
    df_orders = pd.merge(df_orders, df, how='inner', on=col)

In [6]:
def decompose_coordinates(row, prefix = None):
    latitude = float(row.city_coords[1:-1].split(', ')[0])
    longitude = float(row.city_coords[1:-1].split(', ')[1])
    if not prefix:
        return pd.Series({'latitude': latitude, 'longitude': longitude})
    return pd.Series({f"{prefix}_latitude": latitude, f"{prefix}_longitude": longitude})

city2coord = dict()

new_df_dists = df_dists.copy()

for index, row in new_df_dists.iterrows():
    name1 = row["city_from_name"]
    name2 = row["city_to_name"]
    coord1 = row["city_from_coord"]
    coord2 = row["city_to_coord"]
    city2coord[name1] = coord1
    city2coord[name2] = coord2
    

df_cities = pd.DataFrame({
    "city_name":list(city2coord.keys()),
    "city_coords":list(city2coord.values())
})
coords_decomposed = df_cities.apply(lambda x: decompose_coordinates(x), axis=1)
df_cities['latitude'] = coords_decomposed['latitude']
df_cities['longitude'] = coords_decomposed['longitude']
df_cities = df_cities.drop(columns=['city_coords'])

In [7]:
# Join city data
col2coord_prefix = {
    "origin_port":"OP",
    "logistic_hub":"LH",
    "customer":"customer"
}

for col, prefix in col2coord_prefix.items():
    df_orders = pd.merge(df_orders, df_cities, how='left', left_on=col, right_on="city_name")
    df_orders.drop(columns=['city_name'], inplace=True)
    df_orders = df_orders.rename(columns={"latitude":f"{prefix}_latitude", "longitude":f"{prefix}_longitude"})

In [8]:
#AFEGIM DISTANCIES D'ORDRES AMB HUB CENTRAL
df_Hub = df_orders[df_orders.logistic_hub.isna() == False]

df_dists_aux = pd.DataFrame({'city_from_name2':[], 'city_to_name2':[], 'distance2':[]})
df_dists_aux['city_from_name2'] = df_dists['city_from_name']
df_dists_aux['city_to_name2'] = df_dists['city_to_name']
df_dists_aux['distance2'] = df_dists['distance']

df_order_distance_aux = pd.merge(df_Hub,df_dists, how = 'inner', left_on = ['origin_port', 'logistic_hub'], right_on = ['city_to_name', 'city_from_name'])
df_order_distance = pd.merge(df_order_distance_aux,df_dists_aux, how = 'inner', left_on = ['logistic_hub', 'customer'], right_on = ['city_to_name2', 'city_from_name2'])
df_order_distance = df_order_distance.rename(columns = {'distance':'distance1'})

#AFEGIM DISTANCIES D'ORDRES SENSE HUB CENTRAL
df_noHub = df_orders[df_orders.logistic_hub.isna() == True]
df_order_distance3 = pd.merge(df_noHub,df_dists, how = 'inner', left_on = ['origin_port', 'customer'], right_on = ['city_to_name', 'city_from_name'])
df_order_distance3 = df_order_distance3.rename(columns = {'distance':'distance1'})
df_order_distance3['distance2'] = 0

df_order_distance = pd.concat([df_order_distance, df_order_distance3])
df_orders = df_order_distance.drop(columns = ['city_to_name', 'city_from_name', 'city_to_coord', 'city_from_coord', 'city_to_name2', 'city_from_name2'])

In [9]:
df_orders = pd.merge(df_orders, df_products, how='left', on="product_id")

In [10]:
cols2remove = [
    "product_id",
    "order_id",
    "origin_port",
    "3pl",
    "customs_procedures",
    "logistic_hub",
    "customer"
]

df_orders.drop(columns=cols2remove, inplace=True)

In [11]:
df_orders.to_csv("NEW_DATA/train.csv", index=False)

## test.csv

In [12]:
# Refactor variables
def clean_cities(row, col):
    if row[col] == "BCN":
        row[col] = "Barcelona"
    if row[col] == "ATHENAS":
        row[col] = "Athens"
    return row

df_test = df_test.apply(lambda row: clean_cities(row, "origin_port"), axis = 1)
df_test.reset_index(drop=True, inplace=True)
df_test["my_index"] = df_test.index.copy()

In [13]:
# dummies in categorical columns
col2prefix = {
    "origin_port":"OP",
    "3pl":"3pl",
    "customs_procedures":"CP",
    "logistic_hub":"LH",
    "customer":"customer"
}

for col, prefix in col2prefix.items():
    df = pd.get_dummies(df_test[col], prefix=prefix, drop_first=False)
    df = pd.DataFrame(df_test[col]).join(df).drop_duplicates()
    df_test = pd.merge(df_test, df, how='inner', on=col)

In [14]:
def decompose_coordinates(row, prefix = None):
    latitude = float(row.city_coords[1:-1].split(', ')[0])
    longitude = float(row.city_coords[1:-1].split(', ')[1])
    if not prefix:
        return pd.Series({'latitude': latitude, 'longitude': longitude})
    return pd.Series({f"{prefix}_latitude": latitude, f"{prefix}_longitude": longitude})

city2coord = dict()

new_df_dists = df_dists.copy()

for index, row in new_df_dists.iterrows():
    name1 = row["city_from_name"]
    name2 = row["city_to_name"]
    coord1 = row["city_from_coord"]
    coord2 = row["city_to_coord"]
    city2coord[name1] = coord1
    city2coord[name2] = coord2
    

df_cities = pd.DataFrame({
    "city_name":list(city2coord.keys()),
    "city_coords":list(city2coord.values())
})
coords_decomposed = df_cities.apply(lambda x: decompose_coordinates(x), axis=1)
df_cities['latitude'] = coords_decomposed['latitude']
df_cities['longitude'] = coords_decomposed['longitude']
df_cities = df_cities.drop(columns=['city_coords'])

In [15]:
# Join city data
col2coord_prefix = {
    "origin_port":"OP",
    "logistic_hub":"LH",
    "customer":"customer"
}

for col, prefix in col2coord_prefix.items():
    df_test = pd.merge(df_test, df_cities, how='left', left_on=col, right_on="city_name")
    df_test.drop(columns=['city_name'], inplace=True)
    df_test = df_test.rename(columns={"latitude":f"{prefix}_latitude", "longitude":f"{prefix}_longitude"})

In [16]:
#AFEGIM DISTANCIES D'ORDRES AMB HUB CENTRAL
df_Hub = df_test[df_test.logistic_hub.isna() == False]

df_dists_aux = pd.DataFrame({'city_from_name2':[], 'city_to_name2':[], 'distance2':[]})
df_dists_aux['city_from_name2'] = df_dists['city_from_name']
df_dists_aux['city_to_name2'] = df_dists['city_to_name']
df_dists_aux['distance2'] = df_dists['distance']

df_order_distance_aux = pd.merge(df_Hub,df_dists, how = 'inner', left_on = ['origin_port', 'logistic_hub'], right_on = ['city_to_name', 'city_from_name'])
df_order_distance = pd.merge(df_order_distance_aux,df_dists_aux, how = 'inner', left_on = ['logistic_hub', 'customer'], right_on = ['city_to_name2', 'city_from_name2'])
df_order_distance = df_order_distance.rename(columns = {'distance':'distance1'})

#AFEGIM DISTANCIES D'ORDRES SENSE HUB CENTRAL
df_noHub = df_test[df_test.logistic_hub.isna() == True]
df_order_distance3 = pd.merge(df_noHub, df_dists, how = 'inner', left_on = ['origin_port', 'customer'], right_on = ['city_to_name', 'city_from_name'])
df_order_distance3 = df_order_distance3.rename(columns = {'distance':'distance1'})
df_order_distance3['distance2'] = 0

df_order_distance = pd.concat([df_order_distance, df_order_distance3])
df_test = df_order_distance.drop(columns = ['city_to_name', 'city_from_name', 'city_to_coord', 'city_from_coord', 'city_to_name2', 'city_from_name2'])

In [17]:
df_test = pd.merge(df_test, df_products, how='left', on="product_id")

In [18]:
cols2remove = [
    "product_id",
    "origin_port",
    "3pl",
    "customs_procedures",
    "logistic_hub",
    "customer"
]

df_test.drop(columns=cols2remove, inplace=True)

In [19]:
df_test.sort_values(by="my_index", inplace=True)
df_test.drop(columns=["my_index"], inplace=True)

In [20]:
df_test.to_csv("NEW_DATA/test.csv", index=False)